Creator - Sarvesh Joshi 

Date of creation - 14/7/2025

Last updated - 15/7/2025

UPDATE 1 - cleared rounds,leaves,node before fitting and building tree

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
class Node():
    def __init__(self,feature = None ,split_value = None,depth = 0,data=None,Gradients = None, Hessians = None):
        self.feature = feature
        self.split_value = split_value
        self.depth = depth
        self.data = data
        self.left = None
        self.right = None
        self.Gradients = Gradients
        self.Hessians = Hessians 
        self.is_leaf = True if feature ==-1 else False 

    def out(self,lambd,alpha):
        if self.is_leaf == True:
            G = np.sum(self.Gradients)
            H = np.sum(self.Hessians)
    
            if G > alpha:
                return -(G-alpha)/(H+lambd)
            elif G < -alpha:
                return -(G+alpha)/(H+lambd)
            else:
                return 0
    


class Tree():
    def __init__(self,root):
        self.root = root

In [ ]:
class XGBoostBinaryClassifier():
    def __init__(self,max_depth = 10,loss = "logloss",min_split_loss= 0,learning_rate = 1,l2_regularization = 0,max_iter = 1,l1_regularization = 0,min_child_weight=0,subsample = 1):
        self.max_depth = max_depth
        self.loss = loss.lower()
        self.gamma = min_split_loss
        self.lambd = l2_regularization
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.alpha = l1_regularization
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.rounds = []

    
    
    class Round():
        def __init__(self,xgboost_instance,round_num):
            self.xgboost = xgboost_instance
            self.round_num = round_num 
            self.tree = None
            self.Gradients = None
            self.Hessians = None
            self.leaves = []
            self.nodes = []

        def compute_gradients(self,y_train,y_guess):
            if self.xgboost.loss == "logloss":
                y_prob = 1/(1+np.exp(-y_guess))
                G = y_prob-y_train
                H = (y_prob)*(1-y_prob)

            else:
                raise ValueError(f"loss function can only be logloss")
            
            self.Gradients = G
            self.Hessians = H
            #print(self.Hessians)

        def find_split_value(self,X,Gradients,Hessians):
            feature =-1
                # if feature isnt found it stays at -1 and node becomes a leaf
            split_value = 0
            gamma = self.xgboost.gamma
            lambd = self.xgboost.lambd
            min_child_weight = self.xgboost.min_child_weight
            G = np.sum(Gradients)
            H = np.sum(Hessians)
            parent_score = 0.5*((G**2)/(H+lambd))
            max_gain = -np.inf
            

            for curr_feature in range(X.shape[1]):  
                sort_mask = X[:,curr_feature].argsort()
                X_sorted = X[sort_mask]
                G_sorted = Gradients[sort_mask]
                H_sorted = Hessians[sort_mask]
                G_L= 0
                H_L = 0
                for i in range(len(X_sorted)-1):
                    curr_split_value = (X_sorted[i, curr_feature] + X_sorted[i+1, curr_feature]) / 2
                    G_L += G_sorted[i]
                    H_L += H_sorted[i]
                    G_R = G - G_L
                    H_R = H - H_L

                    # if equal values come then split point will be wrongly detected
                    if X_sorted[i,curr_feature] == X_sorted[i+1,curr_feature]:
                        continue
                    
                    curr_gain  = 0.5*((G_L**2)/(H_L + lambd + 1e-12) + (G_R**2)/(H_R+lambd+1e-12)- parent_score) - gamma
                    if (curr_gain >= 0) and (curr_gain > max_gain) :
                        if (H_L >= min_child_weight) and (H_R>=min_child_weight):
                            feature = curr_feature
                            split_value = curr_split_value
                            max_gain = curr_gain
            
            return feature,split_value
        
        def create_node(self,data,depth,Gradients,Hessians):
            feature,split_value = self.find_split_value(data,Gradients=Gradients,Hessians=Hessians)

            
            new_node = Node(feature=feature,split_value=split_value,depth=depth,data=data,Gradients=Gradients,Hessians=Hessians)
            self.nodes.append(new_node)
            return new_node        
        

        def build_tree(self,X):
            
            self.nodes.clear()
            self.leaves.clear()
            curr_depth = 1
            root = self.create_node(data = X,depth=curr_depth,Gradients=self.Gradients,Hessians=self.Hessians)
            #print(root.is_leaf)
            queue = [root]
            leaves = []
            while((curr_depth+1<=self.xgboost.max_depth) and (len(queue)!=0)):
                curr_node = queue.pop(0)
                curr_depth = curr_node.depth
                curr_data = curr_node.data
                curr_Grads = curr_node.Gradients
                curr_Hess = curr_node.Hessians

                if curr_depth+1 == self.xgboost.max_depth:
                    curr_node.is_leaf = True
                    leaves.append(curr_node)

                if not curr_node.is_leaf:
                    mask_left = curr_data[:,curr_node.feature] < curr_node.split_value
                    mask_right = curr_data[:,curr_node.feature] >= curr_node.split_value


                    # Empty masks is creating NaN values ahead !
                    if (np.sum(mask_left)>0 and np.sum(mask_right) > 0) :

                        curr_node.left = self.create_node(depth=curr_depth+1,data = curr_data[mask_left],Gradients= curr_Grads[mask_left],Hessians=curr_Hess[mask_left])
                        curr_node.right = self.create_node(depth=curr_depth+1,data = curr_data[mask_right],Gradients= curr_Grads[mask_right],Hessians=curr_Hess[mask_right])
                        queue.append(curr_node.left)
                        queue.append(curr_node.right)

                    else:
                        curr_node.is_leaf = True
                        leaves.append(curr_node)
                
                elif curr_node.is_leaf:
                    leaves.append(curr_node)
            
            
            self.tree = Tree(root=root)
            self.leaves = leaves


            # for node in self.nodes:
            #     if (node.is_leaf == False) and (node.left == None) and(node.right==None):
            #         print(len(node.data))
        
        def evaluate_tree(self,X,y_guess):
            if self.tree is None or self.tree.root is None:
                raise ValueError("Tree not built yet. Please call fit before predict")
            #print(self.tree.root.left)
            f = []

            for i in range(len(X)):
                curr_node = self.tree.root

                
                while curr_node.is_leaf == False:
                    curr_feature = curr_node.feature
                    curr_split_value = curr_node.split_value

                    if X[i,curr_feature] < curr_split_value:
                        curr_node = curr_node.left
                        
                    else :
                        curr_node = curr_node.right
                    #print(curr_node.is_leaf)
                        
                f.append(curr_node.out(lambd= self.xgboost.lambd,alpha = self.xgboost.alpha))
                #print(curr_node.out(lambd= self.xgboost.lambd,alpha = self.xgboost.alpha))
            f = np.array(f,dtype=float)
            f = f.reshape(y_guess.shape)
            #print(type(f))
            y_guess_new = y_guess + self.xgboost.learning_rate*f
            return y_guess_new
    

    def create_round(self,round_num):
        round = self.Round(xgboost_instance=self,round_num=round_num)
        return round
    
    def fit(self,X_train,y_train):
        if isinstance(X_train,pd.DataFrame):
            X_train = X_train.values
        if (isinstance(y_train,pd.DataFrame)) or (isinstance(y_train,pd.Series)):
            y_train = y_train.values
        y_train = y_train.ravel()
        self.rounds.clear()
        for i in range(self.max_iter):
            round_num = i+1
            self.rounds.append(self.create_round(round_num))
        
        
        
        y_guess = np.zeros(y_train.shape,dtype = float)
        
        
        for round in self.rounds:
            subsample = self.subsample
            num_select = int(len(X_train)*subsample)
            mask = np.random.choice(len(X_train),size=num_select,replace=False)
            X_train_sub = X_train[mask]
            y_train_sub = y_train[mask]
            y_guess_sub = y_guess[mask]
            round.compute_gradients(y_train_sub,y_guess_sub)
            round.build_tree(X_train_sub)
            y_guess = round.evaluate_tree(X_train,y_guess)
            
        
    
    def predict(self,X_test):
        if isinstance(X_test,pd.DataFrame):
            X_test = X_test.values
        y_guess = np.zeros(X_test.shape[0])


        for round in self.rounds:
            y_guess = round.evaluate_tree(X_test,y_guess)
        probs = 1/(1+np.exp(-y_guess))
        
        return (probs>0.5).astype(int)


In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


data = load_breast_cancer()
X = data.data
y = data.target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = XGBoostBinaryClassifier(l2_regularization=0,l1_regularization=0,max_depth=50,max_iter=1)
model.fit(X_train,y_train)
y_pred_manual = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_pred=y_pred_manual,y_true=y_test))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92        43
           1       0.96      0.94      0.95        71

    accuracy                           0.94       114
   macro avg       0.93      0.94      0.93       114
weighted avg       0.94      0.94      0.94       114



In [8]:
import xgboost as xgb
sklearn_XGB = xgb.XGBClassifier()
sklearn_XGB.fit(X_train,y_train)
y_pred_inbuilt = sklearn_XGB.predict(X_test)
print(classification_report(y_pred_inbuilt,y_test))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94        42
           1       0.97      0.96      0.97        72

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



In [9]:
y_pred_inbuilt == y_pred_manual

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])